In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/go-getter/go_getter_25_instruction_sets.csv


In [2]:
# Step 1
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; 
from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"

!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wu713dhy/unsloth_e65cea31a0d14f9c81499cd5864052a0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wu713dhy/unsloth_e65cea31a0d14f9c81499cd5864052a0
  Resolved https://github.com/unslothai/unsloth.git to commit 61dff3514a31b2d86af4b3dcb3fbb07eead7695d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.

In [3]:
# STEP 2
# Import Libraries
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-28 13:58:17.167531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748440697.384596      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748440697.448989      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# STEP 3
# Initialize model

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.5.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [6]:
# STEP 4
# Create PEFT Model 
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    #loftq_cnfig = None, # And LoftQ
)

Unsloth 2025.5.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [31]:
# STEP 5
# Dataset
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
        
    return { "text" : texts, }
	
# Uploaded custom data json into colab's session
# with open('/content/sample_data/final_custom_csv_fnt_data.json', 'r') as f:
with open('/kaggle/input/new-file/Go-Getter_data.txt', 'r') as f:
    json_f = yaml.safe_load(f.read())
	
df = pd.DataFrame(json_f)
print(df.columns)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True,)	

print(dataset[0])

Index(['instruction', 'input', 'output'], dtype='object')


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

{'instruction': "Summarize Chapter 1 of 'The Go-Getter' by Peter B. Kyne.", 'input': 'Cappy Ricks sat at his desk, mulling over how to fill a critical position in his lumber company...', 'output': 'In Chapter 1, Cappy Ricks is introduced as a seasoned businessman seeking a driven individual to manage a new office. The stage is set for a merit-based challenge to find the right candidate.', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize Chapter 1 of 'The Go-Getter' by Peter B. Kyne.\n\n### Input:\nCappy Ricks sat at his desk, mulling over how to fill a critical position in his lumber company...\n\n### Response:\nIn Chapter 1, Cappy Ricks is introduced as a seasoned businessman seeking a driven individual to manage a new office. The stage is set for a merit-based challenge to find the right candidate.<|end_of_text|>"}


In [32]:
# STEP 6
#Train the model

# Set training arguments
training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        #warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Output",
)
print('Done with training_args')

Done with training_args


In [27]:
# print(model, tokenizer, dataset[0])


In [35]:

# Set trainer args with SFT trainer	
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args,
)

print('Done with trainer')

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [ ]:
# Perform training
trainer_stats = trainer.train()


In [ ]:
# STEP 7
# Test the model

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    prompt_format.format(
        "You are an expert at drawing inferences based on your knowledge.", # instruction
        "Based on your knowledge, create the best answer for the question asked on the Go Getter Book. If you do not know then say, you cannot answer the question.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


In [ ]:
outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# STEP 8
# Save locally as sharded model files
model.save_pretrained_merged("amrs_csv_gen_model", 
							tokenizer, 
							save_method = "merged_16bit",
							 )

In [ ]:
# STEP 9
# Push to Huggingface hub - replace the space username as required
model.push_to_hub_merged("amrs-tech/csv_gen_model", 
							tokenizer, save_method = "merged_16bit", 
							token = "hf_HqxQEPdJ******enubpWyh"
							)